In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils
from sklearn.preprocessing import MinMaxScaler
from tensorflow import random, keras
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential

print("Current working directory: {0}".format(os.getcwd()))

## Read data

In [ ]:
RANDOM_SEED = 42
PATH = "data/processed/processed_NE016_10_data.csv"

np.random.seed(RANDOM_SEED)
random.set_seed(RANDOM_SEED)

In [ ]:
df_train, df_val, df_test = utils.read_split_data(PATH)

In [ ]:
train_X = df_train[utils.FEATURE_LIST].fillna(0)
train_y = df_train["targets"].values
test_X = df_test[utils.FEATURE_LIST].fillna(0)
test_y = df_test["targets"].values
val_X = df_val[utils.FEATURE_LIST].fillna(0)
val_y = df_val["targets"].values

## Transform features

In [ ]:
from pickle import dump

scaler = MinMaxScaler().fit(train_X)
dump(scaler, open("models/pump/scaler.pkl", "wb"))


def prepare_Xy(X):
    X = scaler.transform(X)
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    return X


train_X = prepare_Xy(train_X)
test_X = prepare_Xy(test_X)
val_X = prepare_Xy(val_X)

In [ ]:
def plot_signal_hat(Y_test, Y_hat):
    fig = plt.figure()
    plt.plot(Y_hat)
    plt.plot(Y_test)
    plt.legend(["target", "prediction"])
    plt.title("Pediction on test data")
    plt.show()

## Model architecture

In [ ]:
in_shape = train_X.shape

model = Sequential()
model.add(
    LSTM(
        10,
        activation="relu",
        input_shape=(in_shape[1], in_shape[2]),
        return_sequences=True,
    )
)  # ,
# kernel_regularizer = tf.keras.regularizers.L1L2(0.01, 0.01)
# model.add(Dropout(0.3))
model.add(LSTM(10, activation="relu"))
model.add(Dense(units=1))
checkpoint_path = "models/pump/cp.ckpt"
# model.load_weights(checkpoint_path)

model.compile(
    loss="mean_squared_error", optimizer=keras.optimizers.Adam(),
)

model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.01
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.ModelCheckpoint(
        monitor="val_loss", filepath=checkpoint_path, verbose=1,
    ),
]

In [ ]:
round(len(train_X) / 10)

## Model training

In [ ]:
history = model.fit(
    train_X,
    train_y,
    epochs=500,
    batch_size=100,
    validation_data=(val_X, val_y),
    callbacks=callbacks,
)

In [ ]:
model.save("models/pump/")

## Model Prediction

In [ ]:
yhat = model.predict_classes(test_X)

In [ ]:
yhat = yhat / yhat.max()
plot_signal_hat(yhat, test_y)

In [ ]:
yhat[yhat <= 0.5] = 0  # thresolds
yhat[yhat > 0.5] = 1
plot_signal_hat(yhat, test_y)

In [ ]:
df_test["pred_temp"] = yhat
plot_signal_hat(df_test["pred_temp"], df_test["targets"])

In [ ]:
utils.show_metrics(df_test["pred_temp"], df_test["targets"])

## Testing on Abi's data

In [ ]:
df_abi = pd.read_csv("data/processed/abi.csv", header=0)

In [ ]:
abi_X = df_abi[utils.FEATURE_LIST].fillna(0)
abi_Y = df_abi["targets"].values
abi_X = prepare_Xy(abi_X)
yhat2 = model.predict(abi_X)

In [ ]:
yhat2

In [ ]:
yhat2 = yhat2 / yhat2.max()
plot_signal_hat(yhat2[200:1800], abi_Y[200:1800])

In [ ]:
yhat2[yhat2 <= 0.6] = 0  # thresholds
yhat2[yhat2 > 0.6] = 1
plot_signal_hat(yhat2[200:1800], abi_Y[200:1800])

In [ ]:
df_abi["pred_temp"] = yhat2

In [ ]:
plot_signal_hat(
    df_abi["pred_temp"][200:1800], df_abi["Raw Average Velocity (m/s)"][200:1800]
)